In [1]:
from extraction import *
from read_edf import *
from read_result import *
from split_signal import *
from processing_data import *

In [2]:
edf_path = "../database/sleep-cassette/SC4001E0-PSG.edf"
result_path = "../database/sleep-cassette/SC4001EC-Hypnogram.edf"

In [3]:
start_times, durations, labels = read_result_file(result_path)
res = split_edf_by_annotations(edf_path, start_times, durations, labels)
split_signals = split_edf_by_30s(res)
for (_, signals) in split_signals:
    for i in range(3):
        signals[i] = signal_filter(signals[i])

In [4]:
calculate_zero_crossings(signals[0])

275

In [5]:
calculate_hjorth_parameters(signals[0])

(0.2660442890292945, 4.117195702293528)

In [6]:
import numpy as np

In [7]:
np.std(signals[0])

15.541375404107807

In [8]:
calculate_binned_entropy(signals[0], 5)

1.4593447152461914

Page 6, Table 1:

- The binned entropy feature first bins the time series to then sum up the entropy of the bins (this feature is calculated four times for each time a different number of bins; 5, 10, 30, and 60 bins)

- The binned Fourier entropy feature calculates the binned entropy of the power spectral density (this feature is calculated for seven different bin sizes, i.e., 2, 3, 5, 10, 30, 60, and 100 bins)

In [9]:
time_entropy_bins = [5, 10, 30, 60]
frequency_entropy_bins = [2, 3, 5, 10, 30, 60, 100]

In [10]:
frequency_signals = perform_fft(signals[0], 100)
frequency_signals
# print(frequency_signals.shape)

array([85.60199334 -0.j        , 85.81562777 -5.44525069j,
       86.45973896-11.03134076j, ..., 87.52498318+16.90323918j,
       86.45973896+11.03134076j, 85.81562777 +5.44525069j])

In [11]:
res = calculate_frequency_domain_features(frequency_signals, 100)

In [12]:
res

{'spectral_centroid': 5.931166482730863,
 'spectral_variance': 1030.7846397068654,
 'spectral_skewness': 1.471768975949911,
 'spectral_kurtosis': 2.5662569896266065,
 'binned_entropies': [0.037622360322278434,
  0.11471352732076774,
  0.3369915001767694,
  0.8029879670205069,
  1.835945059405911,
  2.6514380495089966,
  3.277695738217079],
 'band_powers': {'delta_slow': 3551091254.3542285,
  'delta_fast': 54489121105.08021,
  'theta': 73289036326.88803,
  'alpha': 86893323356.20265,
  'sigma': 104225869654.38518,
  'beta': 329526840291.22504},
 'power_ratios': {'fast_delta_theta': 127778157431.96823,
  'alpha_theta': 1.1856251318223914,
  'delta_beta': 0.010776333882896732,
  'delta_sigma': 0.03407111177032832,
  'delta_theta': 0.04845323983406528}}

In [14]:
res = signal_extraction(signals[0])

In [15]:
res.shape

(35,)

In [16]:
import os

In [20]:
edf_files = []
result_files = []

for dirname, _, filenames in os.walk('../database/sleep-cassette'):
    for filename in filenames:
        if filename[9] == 'P':
            edf_files.append(os.path.join(dirname, filename))
        else:
            result_files.append(os.path.join(dirname, filename))
N = len(result_files)

X = np.empty((0,18000//down_sampling_rate), float)
Y = np.empty((0,1), str)

print(N)

for i in range(N):
    if i % 10 == 0:
        print(f"{i}-th iteration!")
    result_path = result_files[i]
    edf_path = edf_files[i]
    start_times, durations, labels = read_result_file(result_path)
    res = split_edf_by_annotations(edf_path, start_times, durations, labels)
    split_signals = split_edf_by_30s(res)
    rows_list = []
    for (_, signals) in split_signals:
        three = []
        for i in range(3):
            signals[i] = signal_filter(signals[i])
            tmp_res = signal_extraction(signals[i])
            three.append(tmp_res)
        vec = np.concatenate(three, axis=0)
        rows_list.append(vec)
    cur_X = np.array(rows_list)
    cur_Y = generate_matrix(split_signals)[1]
    # print(cur_X.shape, cur_Y.shape)
    X = np.vstack((cur_X, X))
    Y = np.vstack((cur_Y, Y))
    # print(cur_X.shape)
    # print(cur_Y.shape)
    # break
print(X.shape)
print(Y.shape)

153
0-th iteration!


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 105 and the array at index 1 has size 1800